In [ ]:
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import numpy as np

def calc_ssim(img1_path, img2_path):

    img1 = Image.open(img1_path).convert('L')
    img2 = Image.open(img2_path).convert('L')
    img2 = img2.resize(img1.size)
    img1, img2 = np.array(img1), np.array(img2)
    ssim_score = ssim(img1, img2, data_range=255)
    return ssim_score

img1_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/anya_front.png"
img2_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/front.jpg"
result1 = calc_ssim(img1_path, img2_path)
print(result1)

img1_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/anya_back_rgba.png"
img2_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/back.jpg"
result2 = calc_ssim(img1_path, img2_path)
print(result2)

print((result1 + result2) / 2)

In [ ]:
from skimage.metrics import peak_signal_noise_ratio as psnr
from PIL import Image
import numpy as np


def calc_psnr(img1_path, img2_path):

    img1 = Image.open(img1_path).convert('RGB')
    img2 = Image.open(img2_path).convert('RGB')
    # print(img1.shape)
    # print(img2.shape)
    img2 = img2.resize(img1.size)
    img1, img2 = np.array(img1), np.array(img2)

    # print(img1.shape)
    # print(img2.shape)
    psnr_score = psnr(img1, img2, data_range=255)
    return psnr_score

img1_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/anya_front.png"
img2_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/front3.jpg"
result1 = calc_psnr(img1_path, img2_path)
print(result1)

img1_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/anya_back_rgba.png"
img2_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/back3.jpg"
result2 = calc_psnr(img1_path, img2_path)
print(result2)

print((result1 + result2) / 2)

In [ ]:
import lpips


class util_of_lpips():
    def __init__(self, net, use_gpu=False):

        ## Initializing the model
        self.loss_fn = lpips.LPIPS(net=net)
        self.use_gpu = use_gpu
        if use_gpu:
            self.loss_fn.cuda()

    def calc_lpips(self, img1_path, img2_path):
      

        img1 = Image.open(img1_path).convert('RGB')
        img2 = Image.open(img2_path).convert('RGB')

        img2 = img2.resize(img1.size)
        # img1, img2 = np.array(img1), np.array(img2)

        img1.save("image_evaluate/img1.png")
        img2.save("image_evaluate/img2.png")

        # Load images
        img0 = lpips.im2tensor(lpips.load_image("image_evaluate/img1.png"))  # RGB image from [-1,1]
        img1 = lpips.im2tensor(lpips.load_image("image_evaluate/img2.png"))

        if self.use_gpu:
            img0 = img0.cuda()
            img1 = img1.cuda()
        dist01 = self.loss_fn.forward(img0, img1)
        return dist01


img1_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/anya_front.png"
img2_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/front1.jpg"
temp = util_of_lpips('vgg')
result1 = temp.calc_lpips(img1_path, img2_path)
print(result1)

img1_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/anya_back_rgba.png"
img2_path = "/home/rescue/neon/sfusion/stable-dreamfusion/image_evaluate/back1.jpg"
temp = util_of_lpips('vgg')
result2 = temp.calc_lpips(img1_path, img2_path)
print(result2)

print(((result1 + result2) / 2).detach())

In [ ]:
# example of calculating the frechet inception distance
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy.random import random
from scipy.linalg import sqrtm

# calculate frechet inception distance
def calculate_fid(act1, act2):
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid

# define two collections of activations
act1 = random(10*2048)
act1 = act1.reshape((10,2048))
act2 = random(10*2048)
act2 = act2.reshape((10,2048))
# fid between act1 and act1
fid = calculate_fid(act1, act1)
print('FID (same): %.3f' % fid)
# fid between act1 and act2
fid = calculate_fid(act1, act2)
print('FID (different): %.3f' % fid)